In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 guardrail-ml==0.0.12 tensorboard
!apt-get -qq install poppler-utils tesseract-ocr
!pip install -q unstructured["local-inference"]==0.7.4 pillow

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
from guardrail.client import (
    run_metrics,
    run_simple_metrics,
    create_dataset)

In [ ]:
# Used for multi-gpu
local_rank = -1
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
weight_decay = 0.001
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
max_seq_length = None

# The model that you want to train from the Hugging Face hub
model_name = "Trelis/Llama-2-7b-chat-hf-sharded-bf16"

# Fine-tuned model name
new_model = "llama-2-7b-guanaco-dolly-question-gen"

# The instruction dataset to use
dataset_name = "bnithish/question_difficulty"

# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Number of training epochs
num_train_epochs = 2

# Enable fp16 training, (bf16 to True with an A100)
fp16 = False

# Enable bf16 training
bf16 = False

# Use packing dataset creating
packing = False

# Enable gradient checkpointing
gradient_checkpointing = True

# Optimizer to use, original is paged_adamw_32bit
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "cosine"

# Number of optimizer update steps, 10K original, 20 for demo purposes
max_steps = -1

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 1

# The output directory where the model predictions and checkpoints will be written
output_dir = "./results"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./results/logs"

In [ ]:
def load_model(model_name):
    # Load tokenizer and model with QLoRA configuration
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

    if compute_dtype == torch.float16 and use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
            print("=" * 80)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map=device_map,
        quantization_config=bnb_config
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    # Load LoRA configuration
    peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer, peft_config

In [ ]:
model, tokenizer, peft_config = load_model(model_name)

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00007.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def format_dolly(sample):
    instruction = f"<s>[INST] {sample['instruction']}"
    context = f"Here's some context: {sample['context']}" if len(sample["context"]) > 0 else None
    response = f" [/INST] {sample['response']}"
    # join all the parts together
    prompt = "".join([i for i in [instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# apply prompt template per sample
dataset = load_dataset("bnithish/question_difficulty", split="train")

# Shuffle the dataset
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
#dataset = dataset_shuffled.select(range(50))

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/67 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 67
})

In [ ]:
dataset[0]

{'text': '<s>[INST] What is the difficulty level of this chemistry question- Explain the concept of electronegativity.Here\'s some context: You have the ability to gauage difficulty level of questions. You have easy and difficult questions for reference here: easy question: Define the term "covalent bond" and provide an example.. difficult question: Explain the concept of a chemical equilibrium. [/INST] difficult</s>'}

In [ ]:
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
generator("[INST] Generate a hard Math question in this format:\n ----\n Question: generated question\n ----\n Choices: 1) choice_1 2) choice_2 3)choice_3 4) choice_4\n ----\n [/INST]", max_length=1024)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[{'generated_text': "[INST] Generate a hard Math question in this format:\n ----\n Question: generated question\n ----\n Choices: 1) choice_1 2) choice_2 3)choice_3 4) choice_4\n ----\n [/INST]  Sure! Here's a hard math question in the format you requested:\n\nQuestion: If a bakery sells 250 loaves of bread per day, and each loaf costs $2.50, how much money does the bakery make in a day?\n\nChoices:\n\n1) $500\n2) $625\n3) $750\n4) $875"}]

In [ ]:
'''
### You can try with your own datasets as well
dataset = load_dataset("AlexanderDoria/novel17_test", split="train")
dataset_eval = load_dataset("AlexanderDoria/novel17_test", split="test")

dataset = load_dataset("json", data_files="dolly_llama_formatted_v2 (1).jsonl", split="train")
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
'''

In [ ]:
#dataset = load_dataset("bnithish/question_difficulty", split="train")
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
#dataset = dataset_shuffled.select(range(100))
dataset

Dataset({
    features: ['text'],
    num_rows: 67
})

In [ ]:
dataset[40:45]

{'text': ["<s>[INST] What is the difficulty level of this chemistry question- Define oxidation and reduction.Here's some context: You have the ability to gauage difficulty level of questions. You have easy and difficult questions for reference here: easy question: Write the chemical formula for methane.. difficult question: Explain the concept of electronegativity. [/INST] intermediate</s>",
  "<s>[INST] What is the difficulty level of this chemistry question- What is Avogadro's number?Here's some context: You have the ability to gauage difficulty level of questions. You have easy and difficult questions for reference here: easy question: Write the chemical formula for glucose.. difficult question: What is the significance of the Bohr model of the atom? [/INST] intermediate</s>",
  "<s>[INST] What is the difficulty level of this chemistry question- Explain the difference between an acid and a base.Here's some context: You have the ability to gauage difficulty level of questions. You ha

In [ ]:
prompt = "Here is a difficult level chemistry question starting with what and answer this format:\n ----\n Question: generated question\n ----\n Answer: answer to the question\n ----\n"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, temperature=1)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate a difficult level chemistry question with answer in one of the four choices in this format:
 ----
 Question: generated question
 ----
 Choices: 1) choice_1 2) choice_2 3)choice_3 4) choice_4
 ----
 [/INST]  Sure! Here's a difficult level chemistry question:

Question: What is the major product of the reaction between 2-chloro-3-methylbutane and hydrogen gas at room temperature and atmospheric pressure?

Choices:

1) 2-chloro-3-methylbutanol
2) 2-chloro-3-methylbutene
3) 2-chloro-3-methylbutadiene
4) 2-chloro-3-methylbutane oxide



In [ ]:
prompt = "Here is an easy level chemistry question starting with what and answer this format:\n ----\n Question: generated question\n ----\n Answer: answer to the question\n ----\n"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, temperature=1)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate an easy level chemistry question with answer in one of the four choices in this format:
 ----
 Question: generated question
 ----
 Choices: 1) choice_1 2) choice_2 3)choice_3 4) choice_4
 ----
 [/INST]  Sure! Here's an easy level chemistry question:

Question: What is the chemical formula for sodium chloride?

Choices:

1) NaCl
2) H2O
3) CO2
4) NH3


In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/67 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.506800
2,3.803400
3,3.780600
4,3.833700
5,3.300600
6,3.171800
7,3.233800
8,3.219800
9,2.567000
10,2.578100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Here is a difficult level chemistry question starting with what and answer this format:\n ----\n Question: generated question\n ----\n Answer: answer to the question\n ----\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, temperature = 1, do_sample=True)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Generate a difficult level chemistry question starting with what and answer this format:
 ----
 Question: generated question
 ----
 Answer: answer to the question
 ----
 [/INST] difficulty level: challenging] chemistry- Level difficulty] question: Describe the different types of chemical reactions and provide an example of each.  [/easy] Answer: One of the most basic types of chemical reactions is a synthesis reaction, in which two or moresubstances combine to form a new substance. For example, the reaction between hydrogen gas and oxygen gas to form water is a classic synthesis reaction: Hydrogen gas + Oxygen gas  à H2O (water) [/difficult] chemistry question difficulty level] question: Discuss the concept of electronegativity and its significance in understanding chemical bonding. [/difficult] Answer: Question difficulty level: challenging. chemistry question difficulty level


In [ ]:
prompt = "Here is an easy level chemistry question starting with what and answer this format:\n ----\n Question: generated question\n ----\n Answer: answer to the question\n ----\n"
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Here is an easy level chemistry question starting with what and answer this format:
 ----
 Question: generated question
 ----
 Answer: answer to the question
 ----
 [/INST] easy level chemistry question: What is a chemical reaction? [answer: A chemical reaction is a process in which one or more substances (called reactants) are converted into one or more new substances (called products). This conversion occurs as a result of the interactions between the reactants and a source of energy, such as heat or light. Chemical reactions can produce new substances with different properties than the original reactants.].


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

## Restart runtime to clear VRAM
1. runtime -> Restart runetime
2. Run first three cells at top
3. run the below

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, max_shard_size='2GB')
tokenizer.push_to_hub(new_model)